In [1]:
from sqlalchemy import extract, desc
import datetime
import time
import MetaTrader5 as mt5
import pandas as pd

import options_estrategies as opt_est
from options_estrategies import Option_Due, InformationType
import models as model

In [2]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

estrategies = opt_est.options_estrategies(model, model.PETR4, model.PETR4_OPTIONS)

In [41]:
def tax_convert_dict_from_update_ticks_to_dataframe(tick_data):
    df_tick_data = pd.DataFrame.from_dict(tick_data)
    df_tick_data = df_tick_data.drop(columns=['_sa_instance_state','timestamp_option','days_to_due_date'])
    df_tick_data['stock_price'] = df_tick_data.stock_price.astype(float)
    df_tick_data['exercise_result'] = df_tick_data['strike'] -  df_tick_data['stock_price']  
        
    df_tick_data['tax_exercise'] = (((df_tick_data['strike'] -  df_tick_data['stock_price']) + df_tick_data['bid'])  / df_tick_data['stock_price']) * 100
    df_tick_data['tax'] = (df_tick_data['bid'] / df_tick_data['stock_price'])*100
    df_tick_data = df_tick_data[['option_name','deal_type_zone','stock_price','strike', 'bid','tax_exercise', 'tax']]      
    return df_tick_data


In [42]:
updated_ticks = estrategies.update_quotes(just_last_update=True,mode=InformationType.Real_Time)
ticks_to_process = updated_ticks[0]
options_updated = []
for i in ticks_to_process:
    options_updated.append(i.__dict__)

df_options_updated = convert_dict_from_update_ticks_to_dataframe(options_updated)
df_options_updated.reset_index()
df_options_updated = df_options_updated.set_index('option_name')
df_options_updated

,deal_type_zone,stock_price,strike,bid,TSE,TAXA
option_name,,,,,,
PETRH246,ITM,27.2,25.46,2.21,1.727941,8.125000
PETRH264,ITM,27.2,25.71,2.03,1.985294,7.463235
PETRH229,ITM,27.2,25.96,1.85,2.242647,6.801471
PETRH269,ITM,27.2,26.21,1.71,2.647059,6.286765
PETRH254,ITM,27.2,26.46,1.53,2.904412,5.625000
PETRH274,ITM,27.2,26.71,1.40,3.345588,5.147059
PETRH276,ITM,27.2,26.96,1.30,3.897059,4.779412
PETRH261,OTM,27.2,27.21,1.18,4.375000,4.338235
PETRH256,OTM,27.2,27.46,1.02,4.705882,3.750000
